<a href="https://colab.research.google.com/github/GiliardGodoi/tj-datasets/blob/march-24-version/notebooks/03_verificando_a_ocorr%C3%AAncia_de_certas_express%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive, userdata
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install "tjdatasets @ git+https://github.com/GiliardGodoi/tj-datasets@march-24-version"

  Cloning https://github.com/GiliardGodoi/tj-datasets (to revision march-24-version) to /tmp/pip-install-7cjwa6lo/tjdatasets_a70a78b6afea43b88d561c5cc0dd9358
  Running command git clone --filter=blob:none --quiet https://github.com/GiliardGodoi/tj-datasets /tmp/pip-install-7cjwa6lo/tjdatasets_a70a78b6afea43b88d561c5cc0dd9358
  Running command git checkout -b march-24-version --track origin/march-24-version
  Switched to a new branch 'march-24-version'
  Branch 'march-24-version' set up to track remote branch 'march-24-version' from 'origin'.
  Resolved https://github.com/GiliardGodoi/tj-datasets to commit 2d855d74d0e3802e80998c6dd0f66e236c06ec23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import pandas as pd
import re

from pathlib import Path

In [4]:
from tjdatasets.utils import (
    PATTERN_REMOVE_SPECIAL_CHARS,
    PATTERN_REMOVE_EXTRA_SPACE,
    STANDART_EXPRESSIONS,
    TABLE_REMOVE_LOWER_ACCENTS
    )

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
INPUT_FOLDER = Path(userdata.get('base_folder'))

base_33k_filepath = INPUT_FOLDER / "acordaos_principais_33k.csv"
base_40k_filepath = INPUT_FOLDER / "acordaos_principais_40k.csv"

assert base_33k_filepath.exists()
assert base_40k_filepath.exists()

In [6]:
def preprocessing(df : pd.DataFrame):
    df['formatado'] = (df['conteudo']
                        .str.replace(PATTERN_REMOVE_SPECIAL_CHARS, ' ', regex=True)
                        .str.replace(PATTERN_REMOVE_EXTRA_SPACE, ' ', regex=True)
                        .str.lower()
                        .str.translate(TABLE_REMOVE_LOWER_ACCENTS)
                    )
    return df

In [7]:
def count_expressions(df : pd.DataFrame):
    data = dict()
    for expression in STANDART_EXPRESSIONS.keys():
        data[expression] = df['formatado'].str.count(expression, flags=re.M)

    frame = pd.DataFrame(data)

    return frame

In [8]:
columns =[
    'numero_processo',
    'id_documento',
    'conteudo',
    'codigos_movimentos_temas'
]

filename = base_33k_filepath
# filename = base_40k_filepath

df = pd.read_csv(filename,
                 usecols=columns,
                 encoding='utf8')

In [9]:
frame_33k = (df
             .pipe(preprocessing)
             .pipe(count_expressions))

In [10]:
frame_33k.shape

(33016, 156)

In [22]:
frame_33k.head()

,exmo\.?s?\.?(\s),rel\.,min\.,c\.(?=\s+\w+),dje\.?\s+(?=\d+),"(?:\W+)j\.\s*?\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\W+)p\.?\s*(?=\d+),(\W+)r\.(\W+),(\W+)t\.(\W+),fls(?=\.?\s+\d+),...,"registro:?\s*?\d{1,4}\.\d{5,10}","R\$\s{0,1}[\d\,\.]+","\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\w+)\/(\w+),([a-zA-Z]+)(\d+),"[\d\,\.]+\s{0,1}\%",(\d)\.(\d),(19|20)(\d{2}),\d+(?=\s),\d+$
0,1,0,1,1,1,2,0,5,0,1,...,1,0,11,25,0,0,32,19,28,0
1,1,3,3,1,0,0,0,1,0,4,...,1,0,38,38,0,2,82,44,53,0
2,1,7,7,3,0,7,2,6,0,2,...,1,0,26,74,13,9,85,36,64,0
3,0,18,11,15,4,10,0,4,0,3,...,1,0,56,90,0,10,172,116,93,0
4,0,10,7,7,0,6,0,1,0,1,...,1,0,29,42,0,8,86,48,41,0


In [23]:
frame_33k.describe()

,exmo\.?s?\.?(\s),rel\.,min\.,c\.(?=\s+\w+),dje\.?\s+(?=\d+),"(?:\W+)j\.\s*?\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\W+)p\.?\s*(?=\d+),(\W+)r\.(\W+),(\W+)t\.(\W+),fls(?=\.?\s+\d+),...,"registro:?\s*?\d{1,4}\.\d{5,10}","R\$\s{0,1}[\d\,\.]+","\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\w+)\/(\w+),([a-zA-Z]+)(\d+),"[\d\,\.]+\s{0,1}\%",(\d)\.(\d),(19|20)(\d{2}),\d+(?=\s),\d+$
count,33016.000000,33016.000000,33016.000000,33016.000000,33016.000000,33016.000000,33016.000000,33016.000000,33016.000000,33016.000000,...,33016.000000,33016.0,33016.000000,33016.000000,33016.000000,33016.000000,33016.000000,33016.000000,33016.000000,33016.000000
mean,0.275139,8.274534,6.175612,6.634056,1.190907,5.765538,0.930609,3.587261,0.071571,3.124061,...,0.999849,0.0,33.113642,51.815211,1.188363,7.768718,102.773595,57.832566,61.250061,0.002272
std,0.460548,5.964181,4.098122,4.779483,1.261983,3.942430,2.424508,3.067698,0.413753,3.319696,...,0.014560,0.0,16.752127,27.218140,2.999266,6.292145,52.139139,31.775211,39.004396,0.047608
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,7.000000,4.000000,3.000000,0.000000
25%,0.000000,3.000000,3.000000,2.000000,0.000000,2.000000,0.000000,1.000000,0.000000,1.000000,...,1.000000,0.0,21.000000,33.000000,0.000000,2.000000,66.000000,34.000000,39.000000,0.000000
50%,0.000000,8.000000,6.000000,6.000000,1.000000,6.000000,0.000000,4.000000,0.000000,2.000000,...,1.000000,0.0,33.000000,51.000000,0.000000,8.000000,104.000000,56.000000,52.000000,0.000000
75%,1.000000,12.000000,9.000000,11.000000,2.000000,9.000000,0.000000,4.000000,0.000000,4.000000,...,1.000000,0.0,44.000000,68.000000,0.000000,11.000000,134.000000,77.000000,68.000000,0.000000
max,7.000000,75.000000,53.000000,32.000000,22.000000,57.000000,32.000000,22.000000,15.000000,59.000000,...,2.000000,0.0,186.000000,414.000000,31.000000,88.000000,500.000000,300.000000,398.000000,1.000000


In [24]:
frame_33k[
    frame_33k.sum(axis=1) == 0
]

,exmo\.?s?\.?(\s),rel\.,min\.,c\.(?=\s+\w+),dje\.?\s+(?=\d+),"(?:\W+)j\.\s*?\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\W+)p\.?\s*(?=\d+),(\W+)r\.(\W+),(\W+)t\.(\W+),fls(?=\.?\s+\d+),...,"registro:?\s*?\d{1,4}\.\d{5,10}","R\$\s{0,1}[\d\,\.]+","\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\w+)\/(\w+),([a-zA-Z]+)(\d+),"[\d\,\.]+\s{0,1}\%",(\d)\.(\d),(19|20)(\d{2}),\d+(?=\s),\d+$


In [25]:
sum_33k = frame_33k.sum()

sum_33k[
    sum_33k == 0
]

(\W+)apn(\W+)                                       0
(\W+)ccm(\W+)                                       0
(\W+)dnaee(\W+)                                     0
(\W+)eac(\W+)                                       0
(\W+)eag(\W+)                                       0
(\W+)erms(\W+)                                      0
(\W+)eximp(\W+)                                     0
(\W+)exsusp(\W+)                                    0
(\W+)exverd(\W+)                                    0
(\W+)execar(\W+)                                    0
(\W+)execmc(\W+)                                    0
(\W+)execms(\W+)                                    0
(\W+)hse(\W+)                                       0
(\W+)idc(\W+)                                       0
(\W+)iexecc(\W+)                                    0
(\W+)iuj(\W+)                                       0
(\W+)loman(\W+)                                     0
(\W+)lonmp(\W+)                                     0
(\W+)nc(\W+)                

In [12]:
OUTPUT_FOLDER = Path(userdata.get('base_output'))

dest_filename = OUTPUT_FOLDER / "expressions_33k.csv"

frame_33k.to_csv(dest_filename, index=False)

In [11]:
filename = base_40k_filepath

df = pd.read_csv(filename,
                 usecols=columns,
                 encoding='utf8')

frame_40k = (df.pipe(preprocessing)
                .pipe(count_expressions))

frame_40k.shape

(41374, 156)

In [29]:
dest_filename = OUTPUT_FOLDER / "expressions_40k.csv"

frame_40k.to_csv(dest_filename, index=False)

In [14]:
frame_40k.head()

,exmo\.?s?\.?(\s),rel\.,min\.,c\.(?=\s+\w+),dje\.?\s+(?=\d+),"(?:\W+)j\.\s*?\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\W+)p\.?\s*(?=\d+),(\W+)r\.(\W+),(\W+)t\.(\W+),fls(?=\.?\s+\d+),...,"registro:?\s*?\d{1,4}\.\d{5,10}","R\$\s{0,1}[\d\,\.]+","\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\w+)\/(\w+),([a-zA-Z]+)(\d+),"[\d\,\.]+\s{0,1}\%",(\d)\.(\d),(19|20)(\d{2}),\d+(?=\s),\d+$
0,1,2,1,12,1,1,0,8,1,0,...,1,0,36,51,0,5,58,53,59,0
1,1,1,0,0,0,1,0,4,0,8,...,1,0,8,11,0,5,22,10,19,0
2,0,2,2,1,0,1,2,0,0,4,...,1,0,15,12,0,0,38,19,55,0
3,0,0,0,17,0,0,0,11,0,20,...,1,0,28,41,1,0,45,32,46,0
4,0,0,0,0,0,0,0,0,0,2,...,1,0,6,7,0,2,22,9,18,0


In [15]:
frame_40k.describe()

,exmo\.?s?\.?(\s),rel\.,min\.,c\.(?=\s+\w+),dje\.?\s+(?=\d+),"(?:\W+)j\.\s*?\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\W+)p\.?\s*(?=\d+),(\W+)r\.(\W+),(\W+)t\.(\W+),fls(?=\.?\s+\d+),...,"registro:?\s*?\d{1,4}\.\d{5,10}","R\$\s{0,1}[\d\,\.]+","\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\w+)\/(\w+),([a-zA-Z]+)(\d+),"[\d\,\.]+\s{0,1}\%",(\d)\.(\d),(19|20)(\d{2}),\d+(?=\s),\d+$
count,41374.000000,41374.000000,41374.000000,41374.000000,41374.000000,41374.000000,41374.000000,41374.000000,41374.000000,41374.000000,...,41374.000000,41374.0,41374.000000,41374.000000,41374.000000,41374.000000,41374.000000,41374.000000,41374.000000,41374.0
mean,0.317615,2.062962,0.882752,1.839319,0.717600,0.891067,0.470658,1.820660,0.109006,5.526152,...,0.998163,0.0,16.244574,18.570213,0.614130,2.264731,45.953086,25.679267,35.441703,0.0
std,0.539258,3.592280,2.074264,2.364782,1.948249,1.985822,1.511674,2.265269,0.565964,6.115050,...,0.049619,0.0,13.055965,17.643972,2.103345,4.140055,34.338523,19.390502,25.684082,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,4.000000,3.000000,3.000000,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,...,1.000000,0.0,8.000000,7.000000,0.000000,0.000000,25.000000,14.000000,20.000000,0.0
50%,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,4.000000,...,1.000000,0.0,13.000000,14.000000,0.000000,1.000000,37.000000,21.000000,29.000000,0.0
75%,1.000000,3.000000,1.000000,3.000000,0.000000,1.000000,0.000000,3.000000,0.000000,7.000000,...,1.000000,0.0,20.000000,24.000000,0.000000,3.000000,55.000000,32.000000,44.000000,0.0
max,16.000000,80.000000,44.000000,46.000000,37.000000,37.000000,68.000000,29.000000,29.000000,114.000000,...,4.000000,0.0,642.000000,443.000000,90.000000,146.000000,983.000000,633.000000,674.000000,0.0


In [18]:
frame_40k[
    frame_40k.sum(axis=1) == 0
]

,exmo\.?s?\.?(\s),rel\.,min\.,c\.(?=\s+\w+),dje\.?\s+(?=\d+),"(?:\W+)j\.\s*?\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\W+)p\.?\s*(?=\d+),(\W+)r\.(\W+),(\W+)t\.(\W+),fls(?=\.?\s+\d+),...,"registro:?\s*?\d{1,4}\.\d{5,10}","R\$\s{0,1}[\d\,\.]+","\d{1,2}[\.\/]\d{1,2}[\.\/]\d{2,4}",(\w+)\/(\w+),([a-zA-Z]+)(\d+),"[\d\,\.]+\s{0,1}\%",(\d)\.(\d),(19|20)(\d{2}),\d+(?=\s),\d+$


In [28]:
frame_40k.sum(axis=1).describe()

count    41374.000000
mean       218.762145
std        152.599934
min         27.000000
25%        122.000000
50%        178.000000
75%        267.000000
max       3434.000000
dtype: float64

In [21]:
sum_40k = frame_40k.sum()

sum_40k[
    sum_40k == 0
]

(\W+)erms(\W+)                                      0
(\W+)eximp(\W+)                                     0
(\W+)exsusp(\W+)                                    0
(\W+)exverd(\W+)                                    0
(\W+)execar(\W+)                                    0
(\W+)execmc(\W+)                                    0
(\W+)execms(\W+)                                    0
(\W+)iexecc(\W+)                                    0
(\W+)prcv(\W+)                                      0
(\W+)prpn(\W+)                                      0
(\W+)trbt(\W+)                                      0
(decreto-Lei)\s+(?:n[\.º°]{0,2}\s+)?([\d\/\.-]+)    0
(resolução)\s+(?:n[\.º°]{0,2}\s+)?([\d\/\.-]+)      0
R\$\s{0,1}[\d\,\.]+                                 0
\d+$                                                0
dtype: int64